# Импорт необходимых библиотек

In [45]:
import os
from dotenv import load_dotenv
from minio import Minio
import boto3
from pyspark.sql import SparkSession
import requests
import logging

# Подключение к Minio через библиотеку boto3 

In [31]:
load_dotenv()

# Получаем переменные
endpoint = os.getenv("MINIO_ENDPOINT") # для airflow нужно будет переписать чтобы значения брались из Variables
access_key = os.getenv("MINIO_ACCESS_KEY")
secret_key = os.getenv("MINIO_SECRET_KEY")

# Подключаемся к MinIO
s3 = boto3.client(
    's3',
    endpoint_url=f"http://{endpoint}",
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key

)

# Проверяем список бакетов
try:
    buckets = s3.list_buckets()
    print("Подключение к MinIO через boto3 успешно. Список бакетов:")
    for b in buckets['Buckets']:
        print(f"  - {b['Name']}")
except Exception as e:
    print("Ошибка подключения к MinIO:", e)

Подключение к MinIO через boto3 успешно. Список бакетов:
  - project


# Подключение к Minio через библиотеку Minio 

In [41]:
client = Minio(
    endpoint=os.getenv("MINIO_ENDPOINT"), # для airflow нужно будет переписать чтобы значения брались из Variables
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False  
)

# Проверяем список бакетов
try:
    buckets = client.list_buckets()
    print("Подключение к MinIO через MinIO успешно. Список бакетов:")
    for b in buckets:
        print(f' - {b.name}')
except Exception as e:
    print("Ошибка подключения к MinIO:", e)

Подключение к MinIO через MinIO успешно. Список бакетов:
 - project


# Проверка SparkSession 

In [43]:
spark = SparkSession.builder \
    .appName("MinIO Test") \
    .getOrCreate()

df = spark.range(10)
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



# Получаем данные по API

In [55]:
def get_api_data(env_name):
    load_dotenv()
    # Получаем ссылку из файла .env
    api_url = os.getenv(env_name) # для airflow нужно будет переписать чтобы значения брались из Variables
    # Проверка наличия ссылки
    if not api_url:
        logging.error(f'Указанный URL не задан в файле .env')
        return None
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            logging.info('Данные успешно загружены')
            return data
        else:
            logging.error(f'Error: {response.status_code} - Данные не загружены')
            return None
    except:
        logging.error(f'Error{e} - Данные не загружены')
        raise

get_api_data('CATEGORIES')

{'response': [{'id': 8,
   'title': 'Продукты питания',
   'icon': 'https://rskrf.ru/upload/api/icons/8.png',
   'thumbnail': 'https://rskrf.ru/upload/resize_cache/iblock/069/195_125_240cd750bba9870f18aada2478b24840a/mu8qpc1orxv25xb0qssfguide48perub.png'},
  {'id': 28,
   'title': 'Напитки',
   'icon': 'https://rskrf.ru/upload/api/icons/28.png',
   'thumbnail': 'https://rskrf.ru/upload/resize_cache/iblock/15e/195_125_240cd750bba9870f18aada2478b24840a/50wxtjavm3fsiluopd18lu6k2illkmk9.png'},
  {'id': 11,
   'title': 'Товары для авто',
   'icon': 'https://rskrf.ru/upload/api/icons/11.png',
   'thumbnail': 'https://rskrf.ru/upload/resize_cache/iblock/207/195_125_240cd750bba9870f18aada2478b24840a/97g2rvol7xqyjyz0hxkuj27ch0iq6poq.png'},
  {'id': 5,
   'title': 'Товары для дома',
   'icon': 'https://rskrf.ru/upload/api/icons/5.png',
   'thumbnail': 'https://rskrf.ru/upload/resize_cache/iblock/a3f/195_125_240cd750bba9870f18aada2478b24840a/3fytpy446czj6re3ln6dxndo22lu39i1.png'},
  {'id': 7,
   

# Сохраняем JSON в MinIO